In [438]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from yellowbrick.classifier import confusion_matrix
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [439]:
df = pd.read_csv('../data/socio.csv')

In [440]:
df

,_id,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,quantos_membros,planos_com_ingresso_incluso,estado_civil,risco_de_churn,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao
0,2,52,60,1,1,219,familiar,3,sim,divorciado,alto,1,3,pix,False,2501-5000,VERDADEIRO
1,3,29,20,4,3,435,individual,4,nao,casado,alto,4,5,pix,False,1501-2500,VERDADEIRO
2,5,43,86,3,3,239,individual,1,sim,casado,alto,8,2,boleto,False,0-1500,VERDADEIRO
3,20,54,73,30,8,112,individual,6,sim,divorciado,alto,7,8,cartão de crédito,True,0-1500,VERDADEIRO
4,24,52,15,27,6,111,individual,7,sim,casado,alto,7,4,cartão de débito,True,2501-5000,VERDADEIRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4989,75,74,4,9,289,familiar,5,sim,solteiro,médio,0,0,boleto,False,1501-2500,TRUE
4995,4992,64,23,12,8,447,familiar,7,nao,casado,médio,2,1,cartão de crédito,True,2501-5000,TRUE
4996,4994,8,9,20,5,271,familiar,2,sim,divorciado,médio,4,6,pix,False,0-1500,TRUE
4997,4995,25,55,31,0,263,individual,5,sim,divorciado,médio,4,3,cartão de débito,True,1501-2500,FALSE


In [441]:
df['_id'].value_counts()

_id
2       1
4964    1
4981    1
4980    1
4979    1
       ..
43      1
41      1
40      1
34      1
4999    1
Name: count, Length: 4999, dtype: int64

In [442]:
df.columns

Index(['_id', 'tempo_de_assinatura', 'historico_de_presenca_nos_jogos',
       'historico_de_socio', 'numero_de_desistencias_no_periodo',
       'historico_de_preco_nos_planos', 'tipo_de_plano', 'quantos_membros',
       'planos_com_ingresso_incluso', 'estado_civil', 'risco_de_churn',
       'feedback_sobre_servicos', 'frequencia_de_compra_de_produtos',
       'tipo_pagamento', 'recorrencia', 'renda_mensal', 'aprovacao_cartao'],
      dtype='object')

In [443]:
df['risco_de_churn'] = df['risco_de_churn'].replace('medio', 'médio')
df['risco_de_churn'] = df['risco_de_churn'].replace({'baixo': 0, 'médio': 1, 'alto': 1})

df['risco_de_churn']

/tmp/ipykernel_1896/3704502724.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['risco_de_churn'] = df['risco_de_churn'].replace({'baixo': 0, 'médio': 1, 'alto': 1})


0       1
1       1
2       1
3       1
4       1
       ..
4994    1
4995    1
4996    1
4997    1
4998    1
Name: risco_de_churn, Length: 4999, dtype: int64

In [444]:
df['aprovacao_cartao']=df['aprovacao_cartao'].replace({'VERDADEIRO': True, 'FALSO': False, 'TRUE': True, 'FALSE':False})

/tmp/ipykernel_1896/3522283465.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['aprovacao_cartao']=df['aprovacao_cartao'].replace({'VERDADEIRO': True, 'FALSO': False, 'TRUE': True, 'FALSE':False})


In [445]:
df

,_id,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,quantos_membros,planos_com_ingresso_incluso,estado_civil,risco_de_churn,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao
0,2,52,60,1,1,219,familiar,3,sim,divorciado,1,1,3,pix,False,2501-5000,True
1,3,29,20,4,3,435,individual,4,nao,casado,1,4,5,pix,False,1501-2500,True
2,5,43,86,3,3,239,individual,1,sim,casado,1,8,2,boleto,False,0-1500,True
3,20,54,73,30,8,112,individual,6,sim,divorciado,1,7,8,cartão de crédito,True,0-1500,True
4,24,52,15,27,6,111,individual,7,sim,casado,1,7,4,cartão de débito,True,2501-5000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4989,75,74,4,9,289,familiar,5,sim,solteiro,1,0,0,boleto,False,1501-2500,True
4995,4992,64,23,12,8,447,familiar,7,nao,casado,1,2,1,cartão de crédito,True,2501-5000,True
4996,4994,8,9,20,5,271,familiar,2,sim,divorciado,1,4,6,pix,False,0-1500,True
4997,4995,25,55,31,0,263,individual,5,sim,divorciado,1,4,3,cartão de débito,True,1501-2500,False


In [446]:
media_cac = 3
desvio_padrao_percentual = 0.01
desvio_padrao = media_cac * desvio_padrao_percentual
np.random.seed(42)  # Para garantir reproducibilidade
df['CAC'] = np.random.normal(loc=media_cac, scale=desvio_padrao, size=len(df))

In [447]:
df['LTV'] = (df['tempo_de_assinatura']/10)*df['historico_de_preco_nos_planos']

In [448]:
estatisticas_descritivas = df['tempo_de_assinatura'].describe()

# Acessar os quartis diretamente a partir do resultado
q1 = estatisticas_descritivas['25%']
mediana = estatisticas_descritivas['50%']
q3 = estatisticas_descritivas['75%']

print("Primeiro quartil (Q1):", q1)
print("Mediana:", mediana)
print("Terceiro quartil (Q3):", q3)

Primeiro quartil (Q1): 22.0
Mediana: 43.0
Terceiro quartil (Q3): 65.0


In [449]:
df['expected_payments'] = ((q3 - df['tempo_de_assinatura'])/10) *df['historico_de_preco_nos_planos']

In [450]:
df['ROI'] = (df['LTV']-df['CAC'])/df['CAC']

In [451]:
df.columns

Index(['_id', 'tempo_de_assinatura', 'historico_de_presenca_nos_jogos',
       'historico_de_socio', 'numero_de_desistencias_no_periodo',
       'historico_de_preco_nos_planos', 'tipo_de_plano', 'quantos_membros',
       'planos_com_ingresso_incluso', 'estado_civil', 'risco_de_churn',
       'feedback_sobre_servicos', 'frequencia_de_compra_de_produtos',
       'tipo_pagamento', 'recorrencia', 'renda_mensal', 'aprovacao_cartao',
       'CAC', 'LTV', 'expected_payments', 'ROI'],
      dtype='object')

In [452]:
X = df.drop(['risco_de_churn','_id','estado_civil','renda_mensal'], axis=1)
y=df['risco_de_churn']

In [453]:
# colunas_numericas = X.select_dtypes(include=['float64', 'int64']).columns
# scaler = RobustScaler()
# X[colunas_numericas] = scaler.fit_transform(X[colunas_numericas])
# X

In [454]:
# colunas_categoricas = df.select_dtypes(include=['object']).columns

In [455]:
# ohe = OneHotEncoder()
# X_encoded = ohe.fit_transform(X[colunas_categoricas])
# X_encoded = pd.DataFrame(X_encoded.toarray(), columns=ohe.get_feature_names_out(colunas_categoricas))
# X = pd.concat([X, X_encoded], axis=1).drop(colunas_categoricas, axis=1)

In [456]:
# X

In [457]:
def model_evaluation(y_pred,y_test,y_train,x_train,x_test,model):
    '''
    This function aims to show the main model evaluation metrics (accuracy, recall, precision and f1), as well as the confusion matrix.
    '''
    # Accuracy: The overall percentage of correct predictions made by the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Acuracy: {accuracy:.4f}. (The overall percentage of correct predictions made by the model)')

    # Precision: The ratio of correct positive predictions to the total positive predictions made by the model
    precision = precision_score(y_test, y_pred)
    print(f'Precision: {precision:.4f}(The ratio of correct positive predictions to the total positive predictions made by the model)')

    # Recall: The ratio of correct positive predictions to the total actual positive cases
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall:.4f} (The ratio of correct positive predictions to the total actual positive cases.)')

    # F1-score: A metric that combines precision and recall into a single measure, useful when there is an imbalance between classes
    f1 = f1_score(y_test, y_pred)
    print(f'F1-score: {f1:.4f} ( A metric that combines precision and recall into a single measure, useful when there is an imbalance between classes)')

    #Confusion metrix
    cm = confusion_matrix(model,x_train,y_train,x_test,y_test)
    cm.show;

In [458]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [459]:
# lg= LogisticRegression()
# lg.fit(x_train, y_train)

# # Making predictions
# y_pred = lg.predict(x_test)

# # Evaluating the model
# model_evaluation(y_pred,y_test,y_train,x_train,x_test,lg)

In [460]:

# model_xgb = XGBClassifier(objective='binary:logistic',  random_state=42)

# model_xgb.fit(x_train, y_train)

# y_pred = model_xgb.predict(x_test)

# model_evaluation(y_pred,y_test,y_train,x_train,x_test,model_xgb)

In [461]:
# y_pred = model_xgb.predict_proba(x_test)
# y_pred

In [462]:
# param_grid = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 4, 5],
# }
# xgb_model = XGBClassifier()

# # Crie um objeto GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5)

# # Ajuste o modelo aos dados de treinamento
# grid_search.fit(x_train, y_train)

# # Obtenha os melhores hiperparâmetros encontrados
# best_params = grid_search.best_params_

# # Obtenha o modelo com os melhores hiperparâmetros
# best_xgb_model = grid_search.best_estimator_

# # Faça previsões no conjunto de teste
# y_pred = best_xgb_model.predict(x_test)

# # Avalie o desempenho do modelo
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

# # Exiba os melhores hiperparâmetros encontrados
# print("Melhores hiperparâmetros:")
# print(best_params)

In [463]:
# model_nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# model_nn.fit(x_train, y_train)

# y_pred = model_nn.predict(x_test)

# model_evaluation(y_pred,y_test,y_train,x_train,x_test,model_nn)

In [464]:
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(x_train, y_train)

# # Making predictions
# y_pred = rf.predict(x_test)

# # Evaluating the model
# model_evaluation(y_pred,y_test,y_train,x_train,x_test,rf)

In [465]:
# model_svm = svm.SVC(kernel='linear')
# model_svm.fit(x_train, y_train)

# y_pred = model_svm.predict(x_test)

# model_evaluation(y_pred,y_test,y_train,x_train,x_test,model_svm)

In [466]:
# # Apply Principal Component Analysis (PCA) to reduce dimensionality
# pca = PCA()
# X_pca = pca.fit_transform(X)

# # Retrieve the explained variance ratio for each principal component
# explained_variance_ratio = pca.explained_variance_ratio_

# # Create a DataFrame to store principal component, explained variance ratio, and corresponding feature names
# df_var_exp = pd.DataFrame({'Principal Component': range(1, len(explained_variance_ratio) + 1),
#                             'Explained Variance Ratio': explained_variance_ratio,
#                             'Feature': X.columns})

# # Create a bar plot to visualize the explained variance of each principal component with feature names
# plt.figure(figsize=(12, 8))
# bars = plt.bar(df_var_exp['Principal Component'], df_var_exp['Explained Variance Ratio'], color='blue', alpha=0.7)

# # Add feature names as labels to the bars
# for bar, feature in zip(bars, df_var_exp['Feature']):
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, feature, ha='center', va='bottom', rotation=45, fontsize=8)

# # Set plot labels and title
# plt.xlabel('Principal Component')
# plt.ylabel('Explained Variance Ratio')
# plt.title('Explained Variance of Principal Components with Feature Names')
# plt.show()

In [467]:
# from sklearn.cluster import KMeans
# km = KMeans(n_clusters=5)
# km.fit(X)


In [468]:
# X['cluster']=km.labels_

In [469]:
# X

In [470]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [471]:

# model_xgb = XGBClassifier(objective='binary:logistic',  random_state=42)

# model_xgb.fit(x_train, y_train)

# y_pred = model_xgb.predict(x_test)

# model_evaluation(y_pred,y_test,y_train,x_train,x_test,model_xgb)

In [472]:
# param_grid = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 4, 5],
# }
# xgb_model = XGBClassifier()

# # Crie um objeto GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5)

# # Ajuste o modelo aos dados de treinamento
# grid_search.fit(x_train, y_train)

# # Obtenha os melhores hiperparâmetros encontrados
# best_params = grid_search.best_params_

# # Obtenha o modelo com os melhores hiperparâmetros
# best_xgb_model = grid_search.best_estimator_

# # Faça previsões no conjunto de teste
# y_pred = best_xgb_model.predict(x_test)

# # Avalie o desempenho do modelo
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

# # Exiba os melhores hiperparâmetros encontrados
# print("Melhores hiperparâmetros:")
# print(best_params)

In [473]:
# X.columns

In [474]:
# pca = PCA()
# X_pca = pca.fit_transform(X)

# # Retrieve the explained variance ratio for each principal component
# explained_variance_ratio = pca.explained_variance_ratio_

# # Create a DataFrame to store principal component, explained variance ratio, and corresponding feature names
# df_var_exp = pd.DataFrame({'Principal Component': range(1, len(explained_variance_ratio) + 1),
#                             'Explained Variance Ratio': explained_variance_ratio,
#                             'Feature': X.columns})

# # Create a bar plot to visualize the explained variance of each principal component with feature names
# plt.figure(figsize=(12, 8))
# bars = plt.bar(df_var_exp['Principal Component'], df_var_exp['Explained Variance Ratio'], color='blue', alpha=0.7)

# # Add feature names as labels to the bars
# for bar, feature in zip(bars, df_var_exp['Feature']):
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, feature, ha='center', va='bottom', rotation=45, fontsize=8)

# # Set plot labels and title
# plt.xlabel('Principal Component')
# plt.ylabel('Explained Variance Ratio')
# plt.title('Explained Variance of Principal Components with Feature Names')
# plt.show()

In [475]:
# X.drop(['estado_civil_divorciado',
#        'estado_civil_solteiro', 'cluster'], axis=1,inplace=True)

In [476]:

# model_xgb = XGBClassifier(objective='binary:logistic',  random_state=42)

# model_xgb.fit(x_train, y_train)

# y_pred = model_xgb.predict(x_test)

# model_evaluation(y_pred,y_test,y_train,x_train,x_test,model_xgb)

In [477]:
# param_grid = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 4, 5],
# }
# xgb_model = XGBClassifier()

# # Crie um objeto GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5)

# # Ajuste o modelo aos dados de treinamento
# grid_search.fit(x_train, y_train)

# # Obtenha os melhores hiperparâmetros encontrados
# best_params = grid_search.best_params_

# # Obtenha o modelo com os melhores hiperparâmetros
# best_xgb_model = grid_search.best_estimator_

# # Faça previsões no conjunto de teste
# y_pred = best_xgb_model.predict(x_test)

# # Avalie o desempenho do modelo
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

# # Exiba os melhores hiperparâmetros encontrados
# print("Melhores hiperparâmetros:")
# print(best_params)

In [478]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

In [479]:
# X = df.drop(['risco_de_churn','_id'], axis=1)
# y=df['risco_de_churn']


In [480]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [481]:
# Define a column selector for columns with dtype float64
num_col = make_column_selector(dtype_include=['float64', 'int64'])

# Define a column selector for columns excluding float64 (categorical columns)
cat_col = make_column_selector(dtype_exclude=['float64'])

# Create a column transformer with preprocessing steps
preproc = make_column_transformer(
    (RobustScaler(), num_col),    # Scale numerical columns using RobustScaler
    (OneHotEncoder(handle_unknown='ignore'), cat_col),  # Encode categorical columns using OrdinalEncoder
    remainder='passthrough'        # Pass through columns not specified in the num_col or cat_col
)

# Create a pipeline with the defined preprocessing and Logistic Regression as the estimator
pipe = make_pipeline(preproc, XGBClassifier(objective='binary:logistic' , random_state=42))

In [482]:
# pipe.get_params

In [483]:
# pipe.named_steps.items()

In [484]:
# x_train

In [485]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd419313d00>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd419310c70>)])),
                ('x...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=42, reg_alpha=0,
                               reg_lambda=1, ...))])

In [486]:
# x_test.dtypes

In [487]:
y_pred = pipe.predict(x_test)

In [488]:
param_grid = {
    'xgbclassifier__learning_rate': [0.1, 0.01, 0.001],
    'xgbclassifier__n_estimators': [100, 200, 300],
    'xgbclassifier__max_depth': [3, 4, 5],
}

# Crie um objeto GridSearchCV
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring='accuracy', cv=5)

# Ajuste o modelo aos dados de treinamento
grid_search.fit(x_train, y_train)

# Obtenha os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_

# Obtenha o modelo com os melhores hiperparâmetros
best_xgb_model = grid_search.best_estimator_

# Faça previsões no conjunto de teste
y_pred = best_xgb_model.predict(x_test)

# Avalie o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

# Exiba os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros:")
print(best_params)

Acurácia no conjunto de teste: 0.6440
Melhores hiperparâmetros:
{'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__max_depth': 3, 'xgbclassifier__n_estimators': 300}


In [489]:
modelo = grid_search.best_estimator_
modelo

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd41955e980>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd41955ddb0>)])),
                ('x...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.01, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=3, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=300, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=42, reg_alpha=0,
                               reg_lambda=1, ...))])

In [490]:
import joblib

model_path = '../model/model.pkl'
joblib.dump(modelo, model_path)


['../model/model.pkl']

In [491]:
df

,_id,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,quantos_membros,planos_com_ingresso_incluso,estado_civil,...,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao,CAC,LTV,expected_payments,ROI
0,2,52,60,1,1,219,familiar,3,sim,divorciado,...,1,3,pix,False,2501-5000,True,3.014901,1138.8,284.7,376.723792
1,3,29,20,4,3,435,individual,4,nao,casado,...,4,5,pix,False,1501-2500,True,2.995852,1261.5,1566.0,420.082206
2,5,43,86,3,3,239,individual,1,sim,casado,...,8,2,boleto,False,0-1500,True,3.019431,1027.7,525.8,339.362180
3,20,54,73,30,8,112,individual,6,sim,divorciado,...,7,8,cartão de crédito,True,0-1500,True,3.045691,604.8,123.2,197.575634
4,24,52,15,27,6,111,individual,7,sim,casado,...,7,4,cartão de débito,True,2501-5000,True,2.992975,577.2,144.3,191.851568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4989,75,74,4,9,289,familiar,5,sim,solteiro,...,0,0,boleto,False,1501-2500,True,2.980279,2167.5,-289.0,726.280945
4995,4992,64,23,12,8,447,familiar,7,nao,casado,...,2,1,cartão de crédito,True,2501-5000,True,2.998531,2860.8,44.7,953.067159
4996,4994,8,9,20,5,271,familiar,2,sim,divorciado,...,4,6,pix,False,0-1500,True,3.021342,216.8,1544.7,70.756186
4997,4995,25,55,31,0,263,individual,5,sim,divorciado,...,4,3,cartão de débito,True,1501-2500,False,3.093387,657.5,1052.0,211.550171


In [492]:
novos = pd.read_csv('../data/customers.csv')

In [493]:
novos.head()

,id,name,team_id,email,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,...,estado_civil,prob_de_churn,feedback_sobre_servicos,frequencia_de_compra_de_produtos,created_at_x,updated_at_x,aprovacao_cartao,tipo_pagamento,recorrencia,renda_mensal
0,1,Lissette Goldner,1,hoyt@boyer.example,39,71,11,4,134,familiar,...,solteiro,medio,3,0,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,2501-5000
1,2,Royal Pollich,1,carol_wehner@ernser.test,52,60,1,1,219,familiar,...,divorciado,alto,1,3,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,1501-2500
2,3,Murray Sawayn,1,lionel_cronin@boehm-gottlieb.example,29,20,4,3,435,individual,...,casado,alto,4,5,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,boleto,False,0-1500
3,4,Sydney Wehner,1,lyndsey_wolf@crist.test,15,82,8,1,386,individual,...,solteiro,baixo,2,10,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de crédito,True,0-1500
4,5,Johnsie Ritchie,1,odelia_friesen@ratke.example,43,86,3,3,239,individual,...,casado,alto,8,2,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de débito,True,2501-5000


In [494]:
novos.columns

Index(['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'estado_civil', 'prob_de_churn', 'feedback_sobre_servicos',
       'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x',
       'aprovacao_cartao', 'tipo_pagamento', 'recorrencia', 'renda_mensal'],
      dtype='object')

### Modelo de previsão de tempo de permanência

In [519]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum

def otimizar_retencao(df):
    # Filtre usuários com mais de 70% de probabilidade de sair
    df_retencao = df[df['prob_de_churn']>0.6]

    # Definir o problema de otimização
    model = LpProblem(name="otimizacao_retencao_usuarios", sense=LpMaximize)

    # Variáveis de decisão representando o investimento para reter cada usuário
    investir_retentor = LpVariable.dicts("Investir_Retentor", df_retencao.index, lowBound=0)

    # Função objetivo - maximizar a soma da função desejada para cada usuário
    model += lpSum(df_retencao['LTV'][i] + df_retencao['expected_payments'][i] - df_retencao['CAC'][i] - investir_retentor[i] for i in df_retencao.index)

    # Restrição: o investimento deve ser maior que 10% de (LTV - CAC) para cada usuário
    for i in df_retencao.index:
        model += investir_retentor[i] >= 0.1 * (df_retencao['LTV'][i] - df_retencao['CAC'][i])

    # Restrição: o churn final deve ser inferior ao prob_de_churn para cada usuário
    for i in df_retencao.index:
        model += (df_retencao['prob_de_churn'][i] - 0.01 * investir_retentor[i]) >= 0

    # Restrição: churn final deve estar entre 0.5 e 0.7 para cada usuário
    for i in df_retencao.index:
        model += 0.5 <= (df_retencao['prob_de_churn'][i] - 0.01 * investir_retentor[i]) <= 0.7

    # Resolver o problema de otimização
    model.solve()

    # Adicionar colunas 'investimento_mkt' e 'churn_final' ao DataFrame original
    df['investimento_mkt'] = 0.0
    df['churn_final'] = 0.0

    # Preencher os valores nas colunas correspondentes
    for i in df_retencao.index:
        df.at[i, 'investimento_mkt'] = investir_retentor[i].value()
        df.at[i, 'churn_final'] = df_retencao['prob_de_churn'][i] - 0.01 * investir_retentor[i].value()

    # Exibir resultados
    print("Status:", model.status)

    # Mostrar decisões de investimento para cada usuário
    for i in df_retencao.index:
        print(f"Usuário {i}: Investir = {investir_retentor[i].value()}")

    # Mostrar resultado da função objetivo
    resultado = sum(df_retencao['LTV'][i] + df_retencao['expected_payments'][i] - df_retencao['CAC'][i] - investir_retentor[i].value() for i in df_retencao.index)
    print("Resultado da função objetivo:", resultado)


In [497]:
# customer = pd.read_csv('../data/customers.csv')
# socio = pd.read_csv('../data/socio.csv')

In [498]:
customer = pd.read_csv('../data/customers.csv')
customer = customer.rename(columns={'risco_de_churn': 'prob_de_churn'})
customer.columns

Index(['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'estado_civil', 'prob_de_churn', 'feedback_sobre_servicos',
       'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x',
       'aprovacao_cartao', 'tipo_pagamento', 'recorrencia', 'renda_mensal'],
      dtype='object')

In [499]:
# socio_selecionado = socio[['tipo_pagamento', 'recorrencia', 'renda_mensal']]
# customer = customer.rename(columns={'card_status': 'aprovacao_cartao'})

# merged_df = customer.merge(socio_selecionado, left_index=True, right_index=True)

# merged_df

In [500]:
# final_customer = merged_data[['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
#        'historico_de_presenca_nos_jogos', 'historico_de_socio',
#        'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
#        'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
#        'estado_civil', 'risco_de_churn', 'feedback_sobre_servicos',
#        'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x','card_status']]

In [501]:
customer.to_csv('../data/customers.csv', index=False)#

In [502]:
novos_dados = pd.read_csv('../data/customers.csv')
novos_dados

,id,name,team_id,email,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,...,estado_civil,prob_de_churn,feedback_sobre_servicos,frequencia_de_compra_de_produtos,created_at_x,updated_at_x,aprovacao_cartao,tipo_pagamento,recorrencia,renda_mensal
0,1,Lissette Goldner,1,hoyt@boyer.example,39,71,11,4,134,familiar,...,solteiro,medio,3,0,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,2501-5000
1,2,Royal Pollich,1,carol_wehner@ernser.test,52,60,1,1,219,familiar,...,divorciado,alto,1,3,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,1501-2500
2,3,Murray Sawayn,1,lionel_cronin@boehm-gottlieb.example,29,20,4,3,435,individual,...,casado,alto,4,5,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,boleto,False,0-1500
3,4,Sydney Wehner,1,lyndsey_wolf@crist.test,15,82,8,1,386,individual,...,solteiro,baixo,2,10,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de crédito,True,0-1500
4,5,Johnsie Ritchie,1,odelia_friesen@ratke.example,43,86,3,3,239,individual,...,casado,alto,8,2,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de débito,True,2501-5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4995,Sherryl Zieme,1,claudine@goodwin.test,25,55,31,0,263,individual,...,divorciado,médio,4,3,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,False,boleto,False,1501-2500
4995,4996,Cheryle Konopelski,1,alida.hartmann@hane.example,73,56,5,6,211,individual,...,solteiro,baixo,10,14,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,cartão de crédito,True,2501-5000
4996,4997,The Hon. Jenniffer Labadie,1,celine@cummings.test,1,42,23,3,120,familiar,...,solteiro,alto,9,3,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,pix,False,0-1500
4997,4998,Jesse Botsford Esq.,1,bertram.johnson@mccullough.example,3,41,16,4,64,individual,...,divorciado,baixo,1,5,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,cartão de débito,True,1501-2500


In [503]:
modelo = joblib.load('../model/model.pkl')
novos_dados = pd.read_csv('../data/customers.csv')

colunas_previsao = ['tempo_de_assinatura', 'historico_de_presenca_nos_jogos',
                    'historico_de_socio', 'numero_de_desistencias_no_periodo',
                    'historico_de_preco_nos_planos', 'tipo_de_plano', 'quantos_membros',
                    'planos_com_ingresso_incluso', 'feedback_sobre_servicos',
                    'frequencia_de_compra_de_produtos', 'tipo_pagamento', 'recorrencia',
                    'renda_mensal', 'aprovacao_cartao','LTV', 'CAC', 'expected_payments', 'ROI']


In [504]:
novos_dados.columns

Index(['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'estado_civil', 'prob_de_churn', 'feedback_sobre_servicos',
       'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x',
       'aprovacao_cartao', 'tipo_pagamento', 'recorrencia', 'renda_mensal'],
      dtype='object')

In [505]:
novos_dados['LTV'] = (novos_dados['tempo_de_assinatura']/10)*novos_dados['historico_de_preco_nos_planos']
media_cac = 3
desvio_padrao_percentual = 0.01
desvio_padrao = media_cac * desvio_padrao_percentual
np.random.seed(42)  # Para garantir reproducibilidade
novos_dados['CAC'] = np.random.normal(loc=media_cac, scale=desvio_padrao, size=len(novos_dados))
estatisticas_descritivas = novos_dados['tempo_de_assinatura'].describe()
q3 = estatisticas_descritivas['75%']
novos_dados['expected_payments'] = ((q3 - novos_dados['tempo_de_assinatura'])/10) *novos_dados['historico_de_preco_nos_planos']
novos_dados['ROI'] = (novos_dados['LTV']-novos_dados['CAC'])/novos_dados['CAC']

In [506]:
novos_dados.columns

Index(['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'estado_civil', 'prob_de_churn', 'feedback_sobre_servicos',
       'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x',
       'aprovacao_cartao', 'tipo_pagamento', 'recorrencia', 'renda_mensal',
       'LTV', 'CAC', 'expected_payments', 'ROI'],
      dtype='object')

In [507]:
colunas_nao_utilizadas = novos_dados.columns.difference(colunas_previsao)
colunas_nao_utilizadas

Index(['created_at_x', 'email', 'estado_civil', 'id', 'name', 'prob_de_churn',
       'team_id', 'updated_at_x'],
      dtype='object')

In [508]:
novos_dados[colunas_previsao]

,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,quantos_membros,planos_com_ingresso_incluso,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao,LTV,CAC,expected_payments,ROI
0,39,71,11,4,134,familiar,4,t,3,0,pix,False,2501-5000,True,522.6,3.014901,348.4,172.339001
1,52,60,1,1,219,familiar,3,t,1,3,pix,False,1501-2500,True,1138.8,2.995852,284.7,379.125578
2,29,20,4,3,435,individual,4,f,4,5,boleto,False,0-1500,True,1261.5,3.019431,1566.0,416.793996
3,15,82,8,1,386,individual,4,t,2,10,cartão de crédito,True,0-1500,True,579.0,3.045691,1930.0,189.104649
4,43,86,3,3,239,individual,1,t,8,2,cartão de débito,True,2501-5000,True,1027.7,2.992975,525.8,342.370681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,25,55,31,0,263,individual,5,t,4,3,boleto,False,1501-2500,False,657.5,2.980279,1052.0,219.616942
4995,73,56,5,6,211,individual,4,f,10,14,cartão de crédito,True,2501-5000,True,1540.3,2.998531,-168.8,512.684859
4996,1,42,23,3,120,familiar,3,t,9,3,pix,False,0-1500,True,12.0,3.021342,768.0,2.971745
4997,3,41,16,4,64,individual,4,t,1,5,cartão de débito,True,1501-2500,True,19.2,3.093387,396.8,5.206788


In [509]:
novos_dados.columns

Index(['id', 'name', 'team_id', 'email', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'estado_civil', 'prob_de_churn', 'feedback_sobre_servicos',
       'frequencia_de_compra_de_produtos', 'created_at_x', 'updated_at_x',
       'aprovacao_cartao', 'tipo_pagamento', 'recorrencia', 'renda_mensal',
       'LTV', 'CAC', 'expected_payments', 'ROI'],
      dtype='object')

In [510]:
novos_dados['prob_de_churn'] = modelo.predict_proba(novos_dados[colunas_previsao])[:, 1]

In [511]:
novos_dados

,id,name,team_id,email,tempo_de_assinatura,historico_de_presenca_nos_jogos,historico_de_socio,numero_de_desistencias_no_periodo,historico_de_preco_nos_planos,tipo_de_plano,...,created_at_x,updated_at_x,aprovacao_cartao,tipo_pagamento,recorrencia,renda_mensal,LTV,CAC,expected_payments,ROI
0,1,Lissette Goldner,1,hoyt@boyer.example,39,71,11,4,134,familiar,...,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,2501-5000,522.6,3.014901,348.4,172.339001
1,2,Royal Pollich,1,carol_wehner@ernser.test,52,60,1,1,219,familiar,...,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,pix,False,1501-2500,1138.8,2.995852,284.7,379.125578
2,3,Murray Sawayn,1,lionel_cronin@boehm-gottlieb.example,29,20,4,3,435,individual,...,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,boleto,False,0-1500,1261.5,3.019431,1566.0,416.793996
3,4,Sydney Wehner,1,lyndsey_wolf@crist.test,15,82,8,1,386,individual,...,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de crédito,True,0-1500,579.0,3.045691,1930.0,189.104649
4,5,Johnsie Ritchie,1,odelia_friesen@ratke.example,43,86,3,3,239,individual,...,2024-02-16 20:59:20 UTC,2024-02-16 20:59:20 UTC,True,cartão de débito,True,2501-5000,1027.7,2.992975,525.8,342.370681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4995,Sherryl Zieme,1,claudine@goodwin.test,25,55,31,0,263,individual,...,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,False,boleto,False,1501-2500,657.5,2.980279,1052.0,219.616942
4995,4996,Cheryle Konopelski,1,alida.hartmann@hane.example,73,56,5,6,211,individual,...,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,cartão de crédito,True,2501-5000,1540.3,2.998531,-168.8,512.684859
4996,4997,The Hon. Jenniffer Labadie,1,celine@cummings.test,1,42,23,3,120,familiar,...,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,pix,False,0-1500,12.0,3.021342,768.0,2.971745
4997,4998,Jesse Botsford Esq.,1,bertram.johnson@mccullough.example,3,41,16,4,64,individual,...,2024-02-16 20:59:45 UTC,2024-02-16 20:59:45 UTC,True,cartão de débito,True,1501-2500,19.2,3.093387,396.8,5.206788


In [512]:
novos_dados = pd.concat([novos_dados[colunas_nao_utilizadas], novos_dados[colunas_previsao]], axis=1)

In [513]:
novos_dados.columns

Index(['created_at_x', 'email', 'estado_civil', 'id', 'name', 'prob_de_churn',
       'team_id', 'updated_at_x', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'feedback_sobre_servicos', 'frequencia_de_compra_de_produtos',
       'tipo_pagamento', 'recorrencia', 'renda_mensal', 'aprovacao_cartao',
       'LTV', 'CAC', 'expected_payments', 'ROI'],
      dtype='object')

In [514]:
novos_dados[novos_dados['prob_de_churn']>0.5]

,created_at_x,email,estado_civil,id,name,prob_de_churn,team_id,updated_at_x,tempo_de_assinatura,historico_de_presenca_nos_jogos,...,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao,LTV,CAC,expected_payments,ROI
0,2024-02-16 20:59:20 UTC,hoyt@boyer.example,solteiro,1,Lissette Goldner,0.750436,1,2024-02-16 20:59:20 UTC,39,71,...,3,0,pix,False,2501-5000,True,522.6,3.014901,348.4,172.339001
1,2024-02-16 20:59:20 UTC,carol_wehner@ernser.test,divorciado,2,Royal Pollich,0.676959,1,2024-02-16 20:59:20 UTC,52,60,...,1,3,pix,False,1501-2500,True,1138.8,2.995852,284.7,379.125578
2,2024-02-16 20:59:20 UTC,lionel_cronin@boehm-gottlieb.example,casado,3,Murray Sawayn,0.658229,1,2024-02-16 20:59:20 UTC,29,20,...,4,5,boleto,False,0-1500,True,1261.5,3.019431,1566.0,416.793996
3,2024-02-16 20:59:20 UTC,lyndsey_wolf@crist.test,solteiro,4,Sydney Wehner,0.721181,1,2024-02-16 20:59:20 UTC,15,82,...,2,10,cartão de crédito,True,0-1500,True,579.0,3.045691,1930.0,189.104649
4,2024-02-16 20:59:20 UTC,odelia_friesen@ratke.example,casado,5,Johnsie Ritchie,0.745473,1,2024-02-16 20:59:20 UTC,43,86,...,8,2,cartão de débito,True,2501-5000,True,1027.7,2.992975,525.8,342.370681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,2024-02-16 20:59:45 UTC,claudine@goodwin.test,divorciado,4995,Sherryl Zieme,0.564285,1,2024-02-16 20:59:45 UTC,25,55,...,4,3,boleto,False,1501-2500,False,657.5,2.980279,1052.0,219.616942
4995,2024-02-16 20:59:45 UTC,alida.hartmann@hane.example,solteiro,4996,Cheryle Konopelski,0.667698,1,2024-02-16 20:59:45 UTC,73,56,...,10,14,cartão de crédito,True,2501-5000,True,1540.3,2.998531,-168.8,512.684859
4996,2024-02-16 20:59:45 UTC,celine@cummings.test,solteiro,4997,The Hon. Jenniffer Labadie,0.671631,1,2024-02-16 20:59:45 UTC,1,42,...,9,3,pix,False,0-1500,True,12.0,3.021342,768.0,2.971745
4997,2024-02-16 20:59:45 UTC,bertram.johnson@mccullough.example,divorciado,4998,Jesse Botsford Esq.,0.657580,1,2024-02-16 20:59:45 UTC,3,41,...,1,5,cartão de débito,True,1501-2500,True,19.2,3.093387,396.8,5.206788


In [516]:
novos_dados

,created_at_x,email,estado_civil,id,name,prob_de_churn,team_id,updated_at_x,tempo_de_assinatura,historico_de_presenca_nos_jogos,...,feedback_sobre_servicos,frequencia_de_compra_de_produtos,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao,LTV,CAC,expected_payments,ROI
0,2024-02-16 20:59:20 UTC,hoyt@boyer.example,solteiro,1,Lissette Goldner,0.750436,1,2024-02-16 20:59:20 UTC,39,71,...,3,0,pix,False,2501-5000,True,522.6,3.014901,348.4,172.339001
1,2024-02-16 20:59:20 UTC,carol_wehner@ernser.test,divorciado,2,Royal Pollich,0.676959,1,2024-02-16 20:59:20 UTC,52,60,...,1,3,pix,False,1501-2500,True,1138.8,2.995852,284.7,379.125578
2,2024-02-16 20:59:20 UTC,lionel_cronin@boehm-gottlieb.example,casado,3,Murray Sawayn,0.658229,1,2024-02-16 20:59:20 UTC,29,20,...,4,5,boleto,False,0-1500,True,1261.5,3.019431,1566.0,416.793996
3,2024-02-16 20:59:20 UTC,lyndsey_wolf@crist.test,solteiro,4,Sydney Wehner,0.721181,1,2024-02-16 20:59:20 UTC,15,82,...,2,10,cartão de crédito,True,0-1500,True,579.0,3.045691,1930.0,189.104649
4,2024-02-16 20:59:20 UTC,odelia_friesen@ratke.example,casado,5,Johnsie Ritchie,0.745473,1,2024-02-16 20:59:20 UTC,43,86,...,8,2,cartão de débito,True,2501-5000,True,1027.7,2.992975,525.8,342.370681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,2024-02-16 20:59:45 UTC,claudine@goodwin.test,divorciado,4995,Sherryl Zieme,0.564285,1,2024-02-16 20:59:45 UTC,25,55,...,4,3,boleto,False,1501-2500,False,657.5,2.980279,1052.0,219.616942
4995,2024-02-16 20:59:45 UTC,alida.hartmann@hane.example,solteiro,4996,Cheryle Konopelski,0.667698,1,2024-02-16 20:59:45 UTC,73,56,...,10,14,cartão de crédito,True,2501-5000,True,1540.3,2.998531,-168.8,512.684859
4996,2024-02-16 20:59:45 UTC,celine@cummings.test,solteiro,4997,The Hon. Jenniffer Labadie,0.671631,1,2024-02-16 20:59:45 UTC,1,42,...,9,3,pix,False,0-1500,True,12.0,3.021342,768.0,2.971745
4997,2024-02-16 20:59:45 UTC,bertram.johnson@mccullough.example,divorciado,4998,Jesse Botsford Esq.,0.657580,1,2024-02-16 20:59:45 UTC,3,41,...,1,5,cartão de débito,True,1501-2500,True,19.2,3.093387,396.8,5.206788


In [520]:
otimizar_retencao(novos_dados)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/giovane/.pyenv/versions/lewagon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/db9703945127410790348f177550d21d-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/db9703945127410790348f177550d21d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9197 COLUMNS
At line 21454 RHS
At line 30647 BOUNDS
At line 30648 ENDATA
Problem MODEL has 9192 rows, 3064 columns and 9192 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 1 - largest nonzero change 2.09859e-05 ( 0.00209859%) - largest zero change 0
0  Obj -0 Primal inf 330239.24 (3261)
258  Obj -82940.489 Primal inf

In [521]:
novos_dados.columns

Index(['created_at_x', 'email', 'estado_civil', 'id', 'name', 'prob_de_churn',
       'team_id', 'updated_at_x', 'tempo_de_assinatura',
       'historico_de_presenca_nos_jogos', 'historico_de_socio',
       'numero_de_desistencias_no_periodo', 'historico_de_preco_nos_planos',
       'tipo_de_plano', 'quantos_membros', 'planos_com_ingresso_incluso',
       'feedback_sobre_servicos', 'frequencia_de_compra_de_produtos',
       'tipo_pagamento', 'recorrencia', 'renda_mensal', 'aprovacao_cartao',
       'LTV', 'CAC', 'expected_payments', 'ROI', 'investimento_mkt',
       'churn_final'],
      dtype='object')

In [522]:
novos_dados.sort_values(by='investimento_mkt',ascending=False)

,created_at_x,email,estado_civil,id,name,prob_de_churn,team_id,updated_at_x,tempo_de_assinatura,historico_de_presenca_nos_jogos,...,tipo_pagamento,recorrencia,renda_mensal,aprovacao_cartao,LTV,CAC,expected_payments,ROI,investimento_mkt,churn_final
2548,2024-02-16 20:59:34 UTC,otis.witting@von-kautzer.example,divorciado,2549,Mary Larkin PhD,0.668838,1,2024-02-16 20:59:34 UTC,87,34,...,boleto,False,1501-2500,True,4332.6,3.030641,-1095.6,1428.598322,432.95694,-3.660731
4713,2024-02-16 20:59:43 UTC,terresa@wehner-marvin.example,casado,4714,Celsa Champlin,0.723390,1,2024-02-16 20:59:43 UTC,87,52,...,boleto,False,0-1500,True,4236.9,3.010709,-1071.4,1406.276407,423.38893,-3.510499
3653,2024-02-16 20:59:39 UTC,dion_mueller@schmeler.test,divorciado,3654,Janey Murphy,0.676368,1,2024-02-16 20:59:39 UTC,85,40,...,cartão de crédito,True,1501-2500,True,4216.0,2.975230,-992.0,1416.033119,421.30248,-3.536657
2196,2024-02-16 20:59:33 UTC,kelly_hegmann@dubuque-kunde.test,solteiro,2197,Abram Jenkins DC,0.683151,1,2024-02-16 20:59:33 UTC,83,31,...,pix,False,2501-5000,True,4150.0,3.012777,-900.0,1376.466878,414.69872,-3.463836
2473,2024-02-16 20:59:34 UTC,shirley@towne.example,casado,2474,So Parker,0.683360,1,2024-02-16 20:59:34 UTC,83,70,...,pix,False,1501-2500,True,4100.2,2.958569,-889.2,1384.872856,409.72414,-3.413881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,2024-02-16 20:59:30 UTC,mira@corkery-mann.test,casado,1714,Marlin Feil,0.478329,1,2024-02-16 20:59:30 UTC,45,19,...,cartão de crédito,True,1501-2500,False,1219.5,3.025423,542.0,402.084177,0.00000,0.000000
1712,2024-02-16 20:59:30 UTC,nadene_kuphal@prohaska-pagac.test,casado,1713,Elbert Medhurst I,0.675595,1,2024-02-16 20:59:30 UTC,36,49,...,pix,False,0-1500,True,968.4,2.934820,780.1,328.969139,0.00000,0.675595
1711,2024-02-16 20:59:30 UTC,quyen@dickens.test,solteiro,1712,Ms. Lavon Kiehn,0.677834,1,2024-02-16 20:59:30 UTC,7,1,...,cartão de crédito,True,2501-5000,True,175.7,2.996389,1455.8,57.637255,0.00000,0.677834
1710,2024-02-16 20:59:30 UTC,erin.beahan@weimann.example,divorciado,1711,Heath Frami Jr.,0.628258,1,2024-02-16 20:59:30 UTC,16,39,...,pix,False,0-1500,True,526.4,3.033212,1612.1,172.545379,0.00000,0.628258


In [523]:
novos_dados.drop(['churn_final'],axis=1,inplace=True)
# Salvar o DataFrame resultante no arquivo CSV
novos_dados.to_csv('../data/customers.csv', index=False)
